In [ ]:
# TO DO:
# batch normalization   https://r2rt.com/implementing-batch-normalization-in-tensorflow.html
# batch norm specific for training and testing

# https://gist.github.com/tomokishii/0ce3bdac1588b5cca9fa5fbdf6e1c412
# weight normalization

# optimize the efficiency of codes 

# start with low regularization, large loss
#  more parameters, high learning rate

# reguliarization slows down the convergence rate


# TO DO:
# observe regulairzation and objective losses individually
# orthogonal ini

In [ ]:
# NOTES:

# ---Ini

# Xavier 1/n_in
# He's 2/n_in
# Orthogonal

# ---Activation


# ---Optimizer

# Adam
# Adadelta

# ---Regularization
# dropout + max norm
# batch normalization
# weight normalization

# regularization:  closeness of traning and validation performance, training stableness

In [1]:
import gzip
import os
import tempfile

import numpy as np
import pandas as pd
# from six.moves import urllib
# from six.moves import xrange  # pylint: disable=redefined-builtin


import tensorflow as tf
from tensorflow.contrib.learn.python.learn.datasets.mnist import read_data_sets
from tensorflow.examples.tutorials.mnist import input_data
from tensorflow.examples.tutorials.mnist import mnist
from tensorflow.contrib import rnn

import math
import random

from keras.utils import np_utils

from sklearn import preprocessing

from interNN import *


Couldn't import dot_parser, loading of dot files will not be possible.


Using Theano backend.


In [2]:
# continuous and categorical feature split

# features on columns [0:split_idx) are continuous
# the rest are categorical
def conti_cate_split(dta_df, split_idx):
    
    col_cnt= dta_df.shape[1]
    conti_cols= range(split_idx)
    dis_cols=range(split_idx, col_cnt)
    
    return dta_df[conti_cols], dta_df[dis_cols]

def conti_normalization_train_dta(dta_df):
    
    return preprocessing.scale(dta_df)

def conti_normalization_test_dta(dta_df, train_df):
    
    mean_dim = np.mean(train_df, axis=0)
    std_dim = np.std(train_df, axis=0)
        
    df=pd.DataFrame()
    cols = train_df.columns
    idx=0
    
    for i in cols:
        df[i] = (dta_df[i]- mean_dim[idx])*1.0/std_dim[idx]
        idx=idx+1
        
    return df


In [3]:
# Demographic data
# ------------data prepro-----------------------

files_list=["../dataset/dataset_demo/xtrain.csv", \
            "../dataset/dataset_demo/xtest.csv",\
            "../dataset/dataset_demo/ytrain.csv", \
            "../dataset/dataset_demo/ytest.csv"]

xtrain_df=pd.read_csv( files_list[0] ,sep=',', header=None)
xtest_df=pd.read_csv( files_list[1] ,sep=',', header=None)
ytrain_df=pd.read_csv( files_list[2] ,sep=',', header=None)
ytest_df=pd.read_csv( files_list[3] ,sep=',', header=None)


# get training and testing data prepared

cxtrain, dxtrain = conti_cate_split(xtrain_df, 106)
cxtest, dxtest = conti_cate_split(xtest_df, 106)

cxtest = conti_normalization_test_dta(cxtest, cxtrain)
cxtrain = conti_normalization_train_dta(cxtrain)

# cxtrain = cxtrain.as_matrix() 
dxtrain = dxtrain.as_matrix().astype(int)
cxtest = cxtest.as_matrix()
dxtest = dxtest.as_matrix().astype(int)

ytest = np_utils.to_categorical( ytest_df.as_matrix() )
ytrain =np_utils.to_categorical( ytrain_df.as_matrix() )


print np.shape(cxtrain), np.shape(dxtrain), np.shape(ytrain)
print np.shape(cxtest), np.shape(dxtest), np.shape(ytest)


(6938, 106) (6938, 80) (6938, 8)
(771, 106) (771, 80) (771, 8)


In [3]:
# Default data
# ------------data prepro-----------------------

files_list=["../dataset/dataset_demo/default_xtrain.csv", \
            "../dataset/dataset_demo/default_xtest.csv",\
            "../dataset/dataset_demo/default_ytrain.csv", \
            "../dataset/dataset_demo/default_ytest.csv"]

xtrain_df=pd.read_csv( files_list[0] ,sep=',', header=None)
xtest_df=pd.read_csv( files_list[1] ,sep=',', header=None)
ytrain_df=pd.read_csv( files_list[2] ,sep=',', header=None)
ytest_df=pd.read_csv( files_list[3] ,sep=',', header=None)

# get training and testing data prepared

cxtrain, dxtrain = conti_cate_split(xtrain_df, 20)
cxtest, dxtest = conti_cate_split(xtest_df, 20)

cxtest = conti_normalization_test_dta(cxtest, cxtrain)
cxtrain = conti_normalization_train_dta(cxtrain)

# cxtrain = cxtrain.as_matrix() 
dxtrain = dxtrain.as_matrix().astype(int)
cxtest = cxtest.as_matrix()
dxtest = dxtest.as_matrix().astype(int)

ytest = np_utils.to_categorical( ytest_df.as_matrix() )
ytrain =np_utils.to_categorical( ytrain_df.as_matrix() )

print np.shape(cxtrain), np.shape(dxtrain), np.shape(ytrain)
print np.shape(cxtest), np.shape(dxtest), np.shape(ytest)


(6938, 186) (771, 186) (6938, 1) (771, 1)
(6938, 20) (6938, 166)
(771, 20) (771, 166)


In [6]:
# wide, embedding, co-occurrence and feature interaction 
# version 1: interaction as external hiddens

para_n_epoch = 1200

# tunable parameters

#   representation ability
para_n_hidden_list = [ 32,32,16 ]
#     128,16,8 ]
para_lr = 0.045
para_n_embedding = 2
#   regularization
para_batch_size = 32
para_keep_prob = 1.0
para_l2 = 0.01

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc

# evaluation parameters
para_eval_byepoch = 10


with tf.Session() as sess:
    
    clf = wide_embed_coocc_NN( para_n_conti, para_n_disc, para_n_class, \
                      para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2)
    clf.train_ini()
    clf.inference_ini()
    
    para_cur_lr=para_lr
    
    total_cnt= np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx=range(total_cnt)
    
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
        
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                     para_cur_lr )
        
        if epoch%para_eval_byepoch != 0:
            continue
        
        tmp_test_acc  = clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
        
    print "Optimization Finished!"

In [4]:
# wide, embedding, co-occurrence and feature interaction 
# version 2: individual hidden layers on embedding and interaction

para_n_epoch = 1200

# tunable parameters

#   representation ability
para_n_hidden_list = [ 128,64,32,16 ]
#     128,16,8 ]
para_lr = 0.005
para_n_embedding = 2
#   regularization
para_batch_size = 32
para_keep_prob = 1.0
para_l2 = 0.01

# fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc

# evaluation parameters
para_eval_byepoch = 10


with tf.Session() as sess:
    
    clf = InterNN_IndiH( para_n_conti, para_n_disc, para_n_class, \
                      para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2, para_batch_size)
    clf.train_ini()
    clf.inference_ini()
    
    para_cur_lr=para_lr
    
    total_cnt= np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx=range(total_cnt)
    
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc=0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
        
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                     para_cur_lr )
        
        if epoch%para_eval_byepoch != 0:
            continue
        
        tmp_test_acc  = clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
        
    print "Optimization Finished!"

In [4]:
# v3 fusion
para_n_epoch = 1200

# tunable parameters

# learning speed
para_lr = 0.001
para_batch_size = 256

#   representation ability
para_n_hidden_list = [ 128, 64 ]
#     128,16,8
# !!! change
para_n_embedding = 4


#   regularization
para_l2 = 0.1
# !!!change
para_keep_prob = 1.0
para_max_norm = 7


#  fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc
para_n_coor = 25

# evaluation parameters
para_eval_byepoch = 5

with tf.Session() as sess:
    
    clf = InterNN_fuse( para_n_conti, para_n_disc, para_n_class, \
                        para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2, para_batch_size, para_max_norm, para_n_coor)
    
    
#   change  
    clf.inference_ini()
    clf.train_ini()
    
    
    para_cur_lr = para_lr

    total_cnt   = np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx = range(total_cnt)
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc = 0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
    
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                     para_cur_lr )
        
        if epoch%para_eval_byepoch != 0:
            continue
    
        tmp_test_acc  = clf.inference(dxtest,  cxtest,  ytest,  para_keep_prob) 
        tmp_train_acc = clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, "testing:", tmp_test_acc,\
        "training:", tmp_train_acc
    
    print "Optimization Finished!"

loss on epoch  0  :  1358.97594763 testing: [0.1919585, 3.5016201, 1292.3411] training: [0.21706544, 3.3362319, 1292.3411]
loss on epoch  5  :  881.529536495 testing: [0.26848248, 1.8457818, 841.95215] training: [0.27716923, 1.8004457, 841.95215]
loss on epoch  10  :  582.366710522 testing: [0.27237353, 1.7725637, 556.17255] training: [0.28524071, 1.7447113, 556.17255]
loss on epoch  15  :  386.957800971 testing: [0.27107653, 1.7548887, 369.15247] training: [0.30412224, 1.6888459, 369.15247]
loss on epoch  20  :  257.798899898 testing: [0.21789883, 1.7894963, 245.42532] training: [0.28062841, 1.716526, 245.42532]
loss on epoch  25  :  171.962983308 testing: [0.28664073, 1.6942844, 163.18777] training: [0.31421158, 1.6512724, 163.18777]
loss on epoch  30  :  114.826952051 testing: [0.27367055, 1.7005256, 108.43096] training: [0.31349093, 1.6634634, 108.43096]
loss on epoch  35  :  76.7746836344 testing: [0.2775616, 1.6957891, 71.969795] training: [0.31550878, 1.6374689, 71.969795]
loss 

ERROR: Internal Python error in the inspect module.
Below is the traceback from this internal error.


Unfortunately, your original traceback can not be constructed.



IndexError: string index out of range

In [79]:
def orthogonal(shape):

        flat_shape = (shape[0], np.prod(shape[1:]))
        
        print shape[0],np.prod(shape[1:]), shape[1:]
        
        a = np.random.normal(0.0, 1.0, flat_shape)
        
        u, _, v = np.linalg.svd(a, full_matrices=False)
        q = u if u.shape == flat_shape else v
        
        return q.reshape(shape)
    
tmp = orthogonal([3,5])

print sum( [tmp[0][i]*tmp[0][i] for i in range(5)] )

print sum( [tmp[0][i]*tmp[2][i] for i in range(5)] )

print sum( [tmp[i][0]*tmp[i][0] for i in range(3)] )

print sum( [tmp[i][0]*tmp[i][2] for i in range(3)] )



3 5 [5]
1.0
2.08166817117e-17
0.813187183618
0.194308854259


In [7]:
# fusion, orthogonal ini
para_n_epoch = 1200

# tunable parameters


# learning speed
para_lr = 0.005
para_batch_size = 32

#   representation ability
para_n_hidden_list = [ 128, 64 ]
#  1024, 512, 512
#     128,16,8

# !!! change
para_n_embedding = 3

#   regularization
para_l2 = 0.01

# !!!change
para_keep_prob = 0.8
para_max_norm = 7


#  fixed parameters
para_n_class = 8
para_n_disc = 80
para_n_conti = 106
para_n_disc_voca = [8]*para_n_disc

# evaluation parameters
para_eval_byepoch = 10

with tf.Session() as sess:
    
    clf = InterNN_fuse( para_n_conti, para_n_disc, para_n_class, \
                        para_n_disc_voca, sess, para_n_embedding, para_n_hidden_list,\
                        para_lr, para_l2, para_batch_size, para_max_norm)
    
    
#   change  
    clf.inference_ini()
    clf.train_ini()
    
    
    para_cur_lr = para_lr
    re
    total_cnt   = np.shape(cxtrain)[0]
    total_batch = int(total_cnt/para_batch_size)
    
    total_idx = range(total_cnt)
    
#   training cycle
    for epoch in range(para_n_epoch):
        
        tmpc = 0.0
        
#       shuffle traning instances each epoch  
        np.random.shuffle(total_idx)
    
#       Loop over all batches
        for i in range(total_batch):
            
            batch_idx = total_idx[ i*para_batch_size: (i+1)*para_batch_size ] 
            
            batch_cx = cxtrain[ batch_idx ]
            batch_dx = dxtrain[ batch_idx ]
            batch_y  =  ytrain[ batch_idx ]
            
            tmpc += clf.train_batch( batch_dx, batch_cx, batch_y, para_keep_prob,\
                                     para_cur_lr )
        
        if epoch%para_eval_byepoch != 0:
            continue
    
        tmp_test_acc  = clf.inference(dxtest, cxtest, ytest, para_keep_prob) 
        tmp_train_acc = clf.inference(dxtrain, cxtrain, ytrain, para_keep_prob) 
        
        print "loss on epoch ", epoch, " : ", 1.0*tmpc/total_batch, tmp_test_acc,\
        tmp_train_acc
    
    print "Optimization Finished!"